In [ ]:
import os

import cartopy
import cartopy.crs as ccrs
import geopandas
import matplotlib.pyplot as plt
import numpy as np
import pandas
import rasterio
from rasterio.plot import show as rioshow
import xarray as xr

from ghaa.config import load_config
from ghaa.plot.map import (
    get_axes, plot_basemap, plot_basemap_labels, 
    plot_raster, scale_bar)

In [ ]:
base_path = load_config()["base_path"]
base_path

In [ ]:
color_codes = {
    "ocean": "#bfc0bf",
    "land": "#e4e4e3",
    "flash flooding moderate scenario": "#a2cfe3",
    "flash flooding high scenario": "#0088b7",
    "landslide": "#d5c68e",
    "drought": "",
    "healthcare": "#9f71a4",
    "exposure": "#d53c17",
    "buildings": "#e3e2de",
    "electricity": "#0f6cb2",
    "water": "#87cefa",
    "transport": "#ffd700"
}

In [ ]:
# ax = get_axes()
# plot_basemap(ax, os.path.join(base_path, 'data'), plot_regions=True)
# plot_basemap_labels(ax, os.path.join(base_path, 'data'), include_regions=True)
# scale_bar(ax)

# ax

## Raster plot from access CSV

In [ ]:
# GeoTIFF to plot
tif_path = os.path.join(
    base_path, 'results', 'proximity_results',
    'population_gha_2019-07-01_proximity_downsampled.tif')

In [ ]:
ax = get_axes()
plot_raster(ax, tif_path)

In [ ]:
# Accra
left = -0.5
bottom = 5.4
right = 0.1
top = 5.8
extent = (left, right, bottom, top)
ax = get_axes(extent=extent)
plot_raster(
    ax, tif_path, clip_extent=extent,
    levels=[0, 0.01, 0.1, 1, 10], 
    colors=['#fde725', '#20a378', '#287d8e', '#481567', '#000000'],
)
plot_basemap(ax, os.path.join(base_path, 'data'))

## Alternative methods

In [ ]:
import rioxarray

In [ ]:
ds = rioxarray.open_rasterio(tif_path, mask_and_scale=True)

In [ ]:
with rasterio.open(tif_path) as da:
    crs_code = da.crs.to_epsg()

In [ ]:
dsc = ds.rio.clip_box(
    minx=left,
    miny=bottom,
    maxx=right,
    maxy=top,
)
dsc

In [ ]:
states = geopandas.read_file(
    os.path.join(base_path, 'data', 'admin', 'GHA_admin0.gpkg'))
lakes = geopandas.read_file(
    os.path.join(base_path, 'data', 'nature', 'Polygons', 'GHA_lakes.gpkg'))

In [ ]:
# This is the map projection we want to plot *onto*
# map_proj = ccrs.LambertConformal(central_longitude=-0, central_latitude=5)
map_proj = ccrs.epsg(3857)
ll_proj = ccrs.PlateCarree()

fig, ax = plt.subplots(
    figsize=(10, 10),
    dpi=150,
    subplot_kw={
        'projection': map_proj, # map's projection
    }
)

ax.patch.set_facecolor('#bfc0bf')

ax.set_extent([left, right, bottom, top], crs=ll_proj)
# ax.set_extent((-3.82, 1.82, 4.37, 11.51), crs=ll_proj)


dsc.plot(
    ax=ax,
    levels=[0, 0.01, 0.1, 1, 10], 
    colors=['#fde725', '#20a378', '#287d8e', '#481567', '#000000'],
    transform=ll_proj,  # the data's projection
#     cmap='viridis_r',
)


states.to_crs(map_proj.epsg_code).plot(ax=ax, facecolor='#00000000', edgecolor='#ffffff')
# lakes.to_crs(map_proj.epsg_code).plot(ax=ax, edgecolor='none', facecolor='#87cefa', zorder=1)
